In [1]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import json

In [5]:
today = datetime.today().strftime("%Y%m%d")
base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + today

In [6]:
def get_news_links(page):
    url = base_url + "&page=" + str(page)
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])
    
    return links

def get_news_content(url):
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article',{'id':'dic_area'})
    
    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    else:
        return None, None

In [11]:
# 크롤링 시작
news_links = []
page = 1

while True:
    links = get_news_links(page)
    if not links or any(link in news_links for link in links):
        break
    news_links.extend(links)
    page += 1


In [12]:
# 뉴스 기사 내용 크롤링
news_contents = []
for link in news_links[:20]:    # 숫자 변경
    try:
        title, content = get_news_content(link)
        news_contents.append((title, content))
    except Exception as e:
        print(f"Failed to get content from {link}: {e}")

In [4]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer # pip install sentence_transformers
from bareunpy import Tagger # pip install bareunpy

tagger = Tagger('koba-Q2CYNCI-XZ7E7PI-X6YRKPY-K4Z2KMY')
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


In [39]:
def keyword_ext(text):

    tokenized_doc = tagger.pos(text)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'NNG' or word[1] == 'NNP'])

    n_gram_range = (1,1)

    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)

    return mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    # print(keywords_idx)

    return [words[idx] for idx in keywords_idx]


In [5]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from bareunpy import Tagger
from datetime import datetime

def keyword_ext(text):
    """텍스트에서 키워드를 추출합니다."""
    tokenized_doc = tagger.pos(text)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'NNG' or word[1] == 'NNP'])
    if not tokenized_nouns.strip():
        return []
    
    count = CountVectorizer(ngram_range=(1, 1)).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    if len(candidates) == 0:
        return []

    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)

    return mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
    """다양성을 고려한 최대 마진 적중률(MMR)을 계산하여 키워드를 추출합니다."""
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
    word_similarity = cosine_similarity(candidate_embeddings)

    if len(word_doc_similarity) == 0 or len(word_similarity) == 0:
        return []

    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)
        mmr = (1 - diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)

        if mmr.size == 0:
            break

        mmr_idx = candidates_idx[np.argmax(mmr)]
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

# 뉴스 내용을 포함한 CSV 파일 읽기
today = datetime.today().strftime("%Y%m%d")
# news_contents = pd.read_csv(f"{today}.csv", encoding='utf-8')

news_contents = pd.read_csv("20240701.csv", encoding='utf-8')
news_contents = news_contents.dropna()

# 제목과 키워드를 저장할 DataFrame 생성
df = pd.DataFrame(columns=["Title", "Keywords", "Keyword_Count"])

# 키워드 추출 및 DataFrame에 저장
for i, row in news_contents.iterrows():
    title = row['Title']
    content = row['content']
    if content:
        keywords = keyword_ext(content)
        new_row = pd.DataFrame({"Title": [title], "Keywords": [keywords], "Keyword_Count": [len(keywords)]})
        df = pd.concat([df, new_row], ignore_index=True)

# DataFrame 출력
# print(df)

# 결과를 CSV 파일로 저장
# df.to_csv(f"{today}_keywords.csv", index=False, encoding='utf-8-sig')


# 키워드와 그 빈도를 저장할 DataFrame 생성
all_keywords = []

for keywords in df['Keywords']:
    all_keywords.extend(keywords)

keyword_counts = pd.Series(all_keywords).value_counts().reset_index()
keyword_counts.columns = ['Keywords', 'Count']

# 최종 키워드 빈도수 DataFrame 출력
print(keyword_counts)

# 결과를 CSV 파일로 저장
# keyword_counts.to_csv(f"{today}_keyword_counts.csv", index=False, encoding='utf-8-sig')
keyword_counts.to_csv("20240701_keyword_counts.csv", index=False, encoding='utf-8-sig')

     Keywords  Count
0          서울    792
1          시장    508
2        2024    309
3          가격    297
4          한국    290
...       ...    ...
5618    매봉산의료      1
5619     영주장날      1
5620    박셀바이오      1
5621      김영한      1
5622      독과점      1

[5623 rows x 2 columns]


In [8]:
pd.read_csv("/home/hadoop/project/third_project/data/20240702.csv",encoding='utf-8-sig')

,Title,content
0,'갈길 바쁜데' 총파업 발목잡은 삼성전자 노조..수주 리스크 우려,지난 5월29일 삼성전자 서초사옥 앞에서 전국삼성전자노동조합 파업 선언 기자회견이 ...
1,"트립닷컴, 여름 휴가철 슈퍼세일…도쿄 왕복 항공권 5만원",트립닷컴이 본격적인 여름 휴가철을 맞아 인기 여행지를 위한 할인 프로모션을 시작한다...
2,"공정위, 사행적 판매원 확장한 워너비데이터 시정명령…檢 고발",방문판매법 위반행위 제재…영업정지명령\n\n\n\n공정거래위원회 ⓒ연합뉴스[세종=데...
3,"경기도, 리스로 고가 수입차 타는 고액 체납자 456명 적발",리스보증금 55억원 압류…가택수색·동산압류·형사고발도 추진(수원=연합뉴스) 김경태 ...
4,"‘엔비디아 훈풍’ 단일종목 채권혼합 ETF, 순자산 3배 급증",6개월새 순자산 5100억으로 쑥급등株에 채권혼합 안전성 더해퇴직연금서 '대규모 자...
...,...,...
5331,"DL이앤씨, 6년 연속 건설업계 최고 신용등급 'AA-'","""수익성 중심 내실 경영과 리스크 관리로 기업가치 제고""DL이앤씨가 한국신용평가, ..."
5332,빙그레 더위사냥·생귤탱귤 '제로' 나왔다,[사진=빙그레 제공]빙그레가 당류가 빠진 빙과 신제품 2종을 선보인다고 오늘(2일)...
5333,"‘골든타임을 지키는 다양한 가족’ 배유미 씨, 한미연 영상 공모전 대상","한반도미래인구연구원,‘2024 다양한 가족의 재발견 영상 공모전’ 총 9팀 선정“입..."
5334,"조달청, 올해 1차 품질보증조달물품 지정서 수여…34개사 81개 물품",안전운전 돕는 '교통안전표지'·'정수처리용 산기장치' 등… 납품검사 면제 등 혜택 ...
